In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

In [ ]:
from pathlib import Path
import sys
import yaml
import matplotlib.pyplot as plt
sys.path.append("../../")

In [ ]:
from pytorch_toolbox.core.pipeline import PipelineGraph
from pytorch_toolbox.core.vision.utils import tensor2img
from src.train_with_template import lookups
from src.image import plot_rgby

#### Load the configuration file needed to construct the DAG for the pipeline

In [ ]:
# CONFIG_FILE = Path("../configs/se_resnext50_32x4d_two_input_fc_template.yml")
CONFIG_FILE = Path("../configs/densenet121_two_input_fc_with_tta_template.yml")

In [ ]:
with CONFIG_FILE.open("r") as f:
    config = yaml.load(f)

In [ ]:
pipeline_graph = PipelineGraph.create_pipeline_graph_from_config(config)

In [ ]:
pipeline_graph.sorted_node_names

#### Get the nodes needed to create the learner

In [ ]:
pipeline_graph.run(lookups, to_node="CreateDataBunch")

In [ ]:
create_data_splitter = pipeline_graph.get_node_output("CreateDataSplitter")
create_data_bunch = pipeline_graph.get_node_output("CreateDataBunch")
create_learner = pipeline_graph.get_node_output("CreateLearner")

In [ ]:
train_idx, val_idx = next(iter(create_data_splitter()))

In [ ]:
data_bunch = create_data_bunch(train_idx, val_idx)

#### Uncomment below to see the output of the data loader

In [ ]:
# train_image_batch, _ = next(iter(data_bunch.train_dl))
# train_image = train_image_batch[0]
# plot_rgby(tensor2img(train_image))

In [ ]:
# five_crop_images, _ = next(iter(data_bunch.valid_dl))
# five_crop_image = five_crop_images[0]
# for crop in five_crop_image:
#     plot_rgby(tensor2img(crop))

#### Find the optimal learning rate

In [ ]:
N_ITERS = 333

In [ ]:
learner = create_learner(data_bunch)
learner.lr_find(num_it=N_ITERS)
learner.recorder.plot()